<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Editado por Equipo Docente IIC2233 2019-1 y 2019-2. </font><br>
<font size='1'>Material confeccionado por Fernando Florenzano y Antonio Ossa en 2019.</font>
<br>
</p>

## Ejemplos de estructuras nodales

Los siguientes son ejemplos del uso de estructuras nodales como herramienta de modelación.

### Eficiencia de Listas Ligadas

Una de las ventajas del funcionamiento de Listas Ligadas es que utilizan el **cambio de referencia de objetos** como mecanismo de guardado de datos. Esta es la idea detrás de la implementación de `deque`, que si bien recuerdas, era capaz de extraer rápidamente el primer elemento de la cola, a diferencia de `list`. 

La razón detrás de esto radica en reconocer que `popleft` lo único que realiza para eliminar un elemento es cambiar referencias de objetos, mientras que `list` debe mover todos los elementos contenidos para mantener su estructura, haciendola una operación muy costosa en cuanto a tiempo y recursos de ejecución. Es importante notar que no todo es mejor en estructuras ligadas, la operación de obtener un objeto cualquiera por ejemplo es más costosa en ellas que en `list`. `list` es capaz de acceder directamente a un elemento cualquiera de la lista según su indice, mientras que una cola debe navegar a través de sus elementos para llegar al elemento que se busca.

A continuación se muestra una implementación mediante nodos ligados para una cola, y luego se muestra una prueba de tiempo en contra de `list` para ver que estructura demora más con ciertas operaciones.

In [1]:
class Nodo:
    """Esta clase representa un nodo de una lista ligada"""
    def __init__(self, valor=None):
        """Inicializa la estructura del nodo"""
        self.valor = valor
        self.siguiente = None

class Cola:
    """Clase que representa una lista ligada"""
    
    def __init__(self, iterable=None):
        """Inicializa una cola, con una referencia nula a su cabeza y cola"""
        self.cabeza = None
        self.cola = None
        if iterable is not None:
            for elemento in iterable:
                self.push(elemento)
        
    def push(self, valor):
        """Agrega un nodo al final de la cola"""
        nuevo = Nodo(valor)
        if self.cabeza is None:
            self.cabeza = nuevo
            self.cola = self.cabeza
        else:
            self.cola.siguiente = nuevo
            self.cola = self.cola.siguiente

            
    def obtener(self, posicion):
        """Busca el valor del nodo que está en la posición indicada, partiendo de 0"""
        nodo_actual = self.cabeza
        for _ in range(posicion):
            if nodo_actual is not None:
                nodo_actual = nodo_actual.siguiente
        if nodo_actual is None:
            return None
        return nodo_actual.valor
    
    def popleft(self):
        nodo = self.cabeza
        if self.cabeza is not None:
            self.cabeza = self.cabeza.siguiente
            if self.cabeza is None:
                self.cola = self.cabeza
        return nodo

In [2]:
from time import time

N = 10000000
# Creamos una Cola y una lista con 10000000 elementos
cola = Cola(range(N))
lista = list(range(N))

In [3]:
# Vemos el time actual
tiempo_inicial = time()
# Buscamos el valor intermedio de la estructura, 
cola.obtener(N // 2)
tiempo_final = time()
tiempo_cola = tiempo_final - tiempo_inicial
# Imprimimos el tiempo transcurrido
print(f"""Buscar el elemento {N // 2} en la cola se demoró """
      f"""{tiempo_cola:.6f} segundos.""")

# Vemos el time actual
tiempo_inicial = time()
# Buscamos el valor intermedio de la estructura, 
lista[N // 2]
tiempo_final = time()
tiempo_lista = tiempo_final - tiempo_inicial
# Imprimimos el tiempo transcurrido
print(f"""Buscar el elemento {N // 2} en la lista se demoró """
      f"""{tiempo_lista:.6f} segundos.""")
print(f"El buscar el elemento demoró en la cola {tiempo_cola/tiempo_lista:.2f} veces el tiempo de lista.")

Buscar el elemento 5000000 en la cola se demoró 0.598151 segundos.
Buscar el elemento 5000000 en la lista se demoró 0.000039 segundos.
El buscar el elemento demoró en la cola 15391.58 veces el tiempo de lista.


In [4]:
# Vamos a hacer remover de los primeros 1000 elementos de la lista
N = 1000
tiempo_inicial = time()
for i in range(N):
    lista.pop(0)
tiempo_final = time()
tiempo_lista = tiempo_final - tiempo_inicial
print(f"Sacar los primeros {N} elementos de la lista se demoró {tiempo_lista:.6f} segundos.")

# Vamos a hacer remover de los primeros 1000 elementos de la cola
tiempo_inicial = time()
for i in range(N):
    cola.popleft()
tiempo_final = time()
tiempo_cola = tiempo_final - tiempo_inicial
print(f"Sacar los primeros {N} elementos de la cola se demoró {tiempo_cola:.6f} segundos.")
print(f"La extracción en lista fue {tiempo_lista/tiempo_cola:.2f} veces el tiempo de la cola.")
print()

Sacar los primeros 1000 elementos de la lista se demoró 9.177135 segundos.
Sacar los primeros 1000 elementos de la cola se demoró 0.000569 segundos.
La extracción en lista fue 16125.55 veces el tiempo de la cola.



### Árbol de operaciones

En este ejemplo usaremos la idea árboles como herramienta de modelación para un ejemplo particular.

Trabajaremos con una representación de operaciones matemáticas. Podemos representar una operacion como un árbol donde la raíz contiene la operación o función matemática en sí y esta se aplica sobre sus hijos. 

Por ejemplo, `suma(1, 2)` se representaría con 3 nodos:
* `suma` sería el nodo raíz
* 1 sería un nodo hijo del nodo raíz
* 2 sería otro nodo hijo del nodo raíz

Lo importante en este modealamiento es que los nodos hijos pueden ser otras operaciones, no solamente números. Supongamos que queremos realizar la siguiente operación: `((3+7)*5)*2**1`. Notamos que esto es una composición de distintas operaciones. Primero definiremos cuáles son las operaciones que podemos utilizar:

In [5]:
def sumar(x, y):
    """Suma de x e y, es decir, x + y"""
    return x + y


def multiplicar(x, y):
    """Multiplica a x e y, es decir, x * y"""
    return x * y


def mi_operador(x, y, z):
    """Este es un operador que calcula x*(y^z)"""
    return x * (y ** z)

Ahora, recordemos que para calcular algo como `((3+7)*5)*2**1` necesitamos operar de adentro hacia afuera:
* Primero sumamos 3 + 7
* Este resultado lo multiplicamos por 5
* Y luego a este resultado le aplicamos el operador `mi_operador` junto con 2 y 1 como argumentos.

Como se puede notar, la cantidad de argumentos sobre los que se aplica una operación puede cambiar, por lo que aprovecharemos de ocupar `*args` para recibir cualquier cantidad de argumentos en el constructor de un operador.

In [6]:
class Operacion:

    def __init__(self, operacion, *factores):
        # Operacion es la raiz
        self.operacion = operacion
        # Los factores son los nodos hijos
        self.factores = factores

    def calcular(self):
        factores = []
        # Iteramos sobre cada factor
        for factor in self.factores:
            # Si el factor es una operación
            if type(factor) is Operacion:
                # ... calculamos el resultado de esa operacion
                factor = factor.calcular()
            # Cada factor (ya calculado) lo guardamos para darselo a la funcion
            factores.append(factor)
        # Podemos hacer esto con programacion funcional en una línea!
        # factores = map(lambda f: f.calcular() if type(f) is Operacion else f, self.factores)
        return self.operacion(*factores)

Ahora que tenemos nuestra propia representación de una operación, podemos ocupar el código para resolver `((3+7)*5)*2**1` tal como se comentó antes.

In [7]:
a = Operacion(sumar, 3, 7)  # 10
# a es un arbol con valor "+" y con hijos [3, 7]

b = Operacion(multiplicar, a, 5)  # 10 * 5 = 50
# b es un arbol con valor "*" y con hijos [a, 5]

c = Operacion(mi_operador, b, 2, 1)  # 50 * 2 ** 1 = 100
# c es un arbol con valor "* **" y con hijos [b, 2, 1]

# También podemos ocupar funciones lambda
# Operacion(lambda x, y: x + y, 3, 7)
    
# El resultado deberia ser 100
c.calcular()

100